# Chapter 8. Data Wrangling: Join, Combine and Reshape

In many applications, data may be spread across a number of files or databases or be arranged in a form that is not easy to analyze. This chapter focuses on tools to help combine, join, and rearrange data. 

First, I introduce the concept of hierarchical indexing in pandas, which is used extensively in some of these operations. I then dig into the particular data manipulations. You can see various applied usages of these tools in Chapter 14

## Hierarchichal indexing
This is a feature that enables you to have multiple index levels on an axis. Somewhat abstractly, it proved a way for you to work with higher dimensional data in a lower dimensional form. Let's start with a simple example; create e Series with a list of lists (or arrays) as the index:

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.Series(np.random.randn(9), 
                 index = [['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                          [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -2.066670
   2    0.045779
   3   -0.612050
b  1    1.132699
   3    2.039230
c  1    0.526458
   2    1.262576
d  2   -0.204149
   3   -0.459534
dtype: float64

What you are seeing is a prettified view of a Series with a MultiIndex as its index. The "gaps" in the index displays mean "use the label directly above"

In [4]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

With a hierarchically indexed object, so-called partial indexing is possible, enabling you to concisely select subsets of the data:

In [5]:
data['b']

1    1.132699
3    2.039230
dtype: float64

In [6]:
data['b':'c']

b  1    1.132699
   3    2.039230
c  1    0.526458
   2    1.262576
dtype: float64

In [7]:
data.loc[['b', 'd']]

b  1    1.132699
   3    2.039230
d  2   -0.204149
   3   -0.459534
dtype: float64

Hierarchical data plays an important role in reshaping data and group based operations like forming a pivot table. For example, you could rearrange the data into a DataFrame using its unstack method:

In [8]:
data.unstack()

,1,2,3
a,-2.066670,0.045779,-0.612050
b,1.132699,NaN,2.039230
c,0.526458,1.262576,NaN
d,NaN,-0.204149,-0.459534


In [9]:
data.unstack().stack()

a  1   -2.066670
   2    0.045779
   3   -0.612050
b  1    1.132699
   3    2.039230
c  1    0.526458
   2    1.262576
d  2   -0.204149
   3   -0.459534
dtype: float64

With a DataFrame, either axis can have a hierarchical index:

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)), 
                     index = [['a', 'a', 'b', 'b'],
                              [1, 2, 1, 2,]],
                     columns = [['Ohio', 'Ohio', 'Colorado'],
                                ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [11]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

## Reordering and Sorting Levels

At times you will need t rearrange the order of the levels on an axis or sort the data by the values in one specific level. The swaplevel takes two level numbers or names and returns a new object with the levels interchanged (but the data is otherwise unaltered):

In [12]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

*sort_index* on the other hand, sorts the data using only the values in a single level. When swapping lveles, it's not uncommon to also use sort_index so that the result is lexicographically sorted by the indicated level:

In [13]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

### Summary statistics by level

Many descriptive and summary statitics on DataFrame and Series have a *level* option in which you can specify the level you want to aggregate by in a particular axis. Consider the above DataFrame; we can aggregate by level on either the rows or columns like so:

In [14]:
frame.sum(level = 'key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [15]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's columns

It's not unusual to want to use one or more columns from a DataFrame as the row index; alternatively, you may wish to move the row index into the DataFrame's columns. Here's an example DataFrame:

In [16]:
frame = pd.DataFrame({'a' : range(7),
                      'b' : range(7, 0, -1),
                      'c' : ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd' : [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame's set_index function will create a new DataFrame using one or more of its columns as the index:

In [18]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

## Combining and Merging Datasets

Data contained in pandas object can be combined together in a number of ways:

* *pandas.merge* connects rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases as it implements database *join* operations.

* pandas.concat concatenates or "stacks" togheter objects along an axis.

* The combine_first instance method enables splicing together overlapping data to fill in missing values in one object with values from another.

### Database-Style DataFrame Joins

*Merge* or *join* operations combine dataset by linking rows using one or more keys. These operations are central to relational databases. 

In [19]:
df1 = pd.DataFrame({'key' : ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data' : range(7)})
df2 = pd.DataFrame({'key' : ['a', 'b', 'd'],
                    'data2' : range(3)})
df1

,key,data
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [20]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [21]:
pd.merge(df1, df2)

,key,data,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


It is good practice to specify the key:

In [22]:
pd.merge(df1, df2, on='key')

,key,data,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If the column names are different in each object, you can specify them seperately:

In [24]:
df3 = pd.DataFrame({'lkey' : ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1' : range(7)})
df4 = pd.DataFrame({'rkey' : ['a', 'b', 'd'],
                    'data2' : range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


This is the inner join or $$P(D_3 \cap D_4)$$

will exclude 'c' and 'd' values. To include these you may use outer join or other kind of joins.

Many-to-many merges have well-defined, though not necessarily intuitive behavior. Here's an example

In [29]:
df1 = pd.DataFrame({'key' : ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1' : range(6)})
df2 = pd.DataFrame({'key' : ['a', 'b', 'a', 'b', 'd'],
                    'data1' : range(5)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [30]:
df2

,key,data1
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [31]:
pd.merge(df1, df2, on='key', how='left')

,key,data1_x,data1_y
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


Many to many joins form the cartesian product of the rows. Since there were three 'b' rows in the left DataFrame and two in the right, there are six 'b' rows in the result. The join method only affects the distinct key values appearing in the result:

In [32]:
pd.merge(df1, df2, how='inner')

,key,data1
0,b,1
1,a,2


A last issue to consider in merge operations is the treatment of overlapping column names. While you can address the overlap manually (see the earlier section on renaming axis labels), merge has a suffixes ption for specifying strings to append to overlapping names in the left and right DataFrame objets:

### Merging on Index

In some cases, the merge key(s) in a DataFrame will be found in its index. In this case, you can pass left_index = True or right_index = True (or both) to indicate that the index should be used as the merge key:

In [38]:
left1 = pd.DataFrame({'key' : ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value' : range(6)})
right1 = pd.DataFrame({'group_val' : [3.5, 7]},
                     index = ['a', 'b'])
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [39]:
right1

,group_val
a,3.5
b,7.0


In [40]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


Since the default merge method is to intersect the join keys, you can instead form the union of them with an outer join:

In [41]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


With hierarchically indexed data, things are more complicated, as joining on index is implicitly a multiple-key merge:

In [48]:
lefth = pd.DataFrame({'key1' : ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                      'key2' : [2000, 2001, 2002, 2001, 2002],
                      'data' : np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)), 
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                              [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
lefth      

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [49]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In this case, you have to indicate multiple columns to merge on as a list (note the handling of duplicate index values with *how='outer'*)

In [51]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True) 

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


### Concatenating Along an Axis

Another kind of data combination operation is referred to interchangeably as concatenation, binding or stacking. Numpy's concatenate function can do this with NumPy arrays:

In [53]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [54]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In the context of pandas object such as Series and DataFrame, having labeled axes enable you ro furhter generalize array concatenation. In particular, you have a number of additional things to think about:

* If the objects are indexed differently on the other axes, should we combine the distinct elements in these axes or use only the shared values (the intersection)?
* Do the concatenated chunks of data need to be identifiable in the resulting object?
* Does the "concatenation axis" contain data that needs to be preserved? In many cases, the default integer labels in a DataDrame are best discarded during concatenation.

The concat function in pandas provides a consistent way to address each of these concerns. Ill give a number of examples to illustrate how i works. Suppose we have three Series with no index overlap:

In [56]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

calling concat with these objects in a list glues together the values and indexes:

In [57]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

By default concat works axis=0, producing another Series. If you pass axis=1, the result will instead be a DataFrame (axis=1 is the columns):

In [58]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In this case there is no overlap on the other axis, which you can see is the sorted union. You can instead intersect them by passing *join='inner'*

In [60]:
s4 = pd.concat([s1, s2])
s4

a    0
b    1
c    2
d    3
e    4
dtype: int64

In [61]:
pd.concat([s1, s4], axis = 1)

,0,1
a,0.0,0
b,1.0,1
c,NaN,2
d,NaN,3
e,NaN,4


In [62]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


A potential issue is that the concatenated pieces are not identifiable in the result. Suppose instead you wanted to create a hierarchical index on the concatenation axis. To do this, use the keys argument:

In [64]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [65]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In the case of combining Series along axis=1, the keys becom the DataFrame column headers:

In [66]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [ ]:
The same logic extends to dataframe objects.

### Combing Data with Overlap

There is another data combination situation that can't be expressed as either a merge or concatenation operation. You may have two datasets whose indexes overlap in full or part. As a motivating example, consider NumPy's where function, which performs the array-oriented equivalent of an if-else expression:

In [68]:
a = pd.Series(
    [np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
    index=['f', 'e', 'd', 'c', 'b', 'a']
)

b = pd.Series(
    [0., np.nan, 2., np.nan, np.nan, 5.],
    index=['a', 'b', 'c', 'd', 'e', 'f']
)

a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [69]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [70]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

Series have a combine_first method, which perform the equivalent of this operation along with pandas usual data alignment logic:

In [71]:
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

## Reshaping and Pivoting

There are a number of basic operations for rearranging tabular data. These are alternatingly referred to as reshape or pivot operations.

### Reshaping with Hierarchichal Indexing

Hierarchical indexing provides a consistent way to rearrange data in a DataFrame. There are two primary actions:

*stack*
this rotates or pivots from the columns in the data to the rows

*unstack*
this pivots from the row into the columns

Ill illustrate these operations through a series of examples. Consider a small DataFrame with string arrays as row and column indexes:

In [74]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'), 
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Using the stack method on this data pivots the columns into the rows, producing a Series:

In [75]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32